In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings 
warnings.filterwarnings("ignore")

In [46]:
df = pd.read_csv('SpotifyFeatures.csv')
df.head()

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.611,0.389,99373,0.910,0.000,C#,0.3460,-1.828,Major,0.0525,166.969,4/4,0.814
1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.246,0.590,137373,0.737,0.000,F#,0.1510,-5.559,Minor,0.0868,174.003,4/4,0.816
2,Movie,Joseph Williams,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,0.952,0.663,170267,0.131,0.000,C,0.1030,-13.879,Minor,0.0362,99.488,5/4,0.368
3,Movie,Henri Salvador,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.703,0.240,152427,0.326,0.000,C#,0.0985,-12.178,Major,0.0395,171.758,4/4,0.227
4,Movie,Fabien Nataf,Ouverture,0IuslXpMROHdEPvSl1fTQK,4,0.950,0.331,82625,0.225,0.123,F,0.2020,-21.150,Major,0.0456,140.576,4/4,0.390


# Model fitting


In [4]:
features = ["acousticness", "duration_ms", "energy", "instrumentalness", "key", "mode", "speechiness", "tempo", "valence"]

In [5]:
y = df['popularity']
y

0          0
1          1
2          3
3          0
4          4
          ..
232720    39
232721    38
232722    47
232723    44
232724    35
Name: popularity, Length: 232725, dtype: int64

**Using replace for mode column**(as there are only two values)

In [6]:
df['mode'] = df['mode'].str.replace("Major","1")
df['mode'] = df['mode'].str.replace("Minor","0")
df['mode'] = df['mode'].astype('int')

In [7]:
drop_feas=[]
for i in df.columns:
    if i not in features:
        drop_feas.append(i)

In [8]:
df.drop(drop_feas,axis=1,inplace=True)


**Using LabelEncoder for key column**

In [9]:
df['key'].unique()

array(['C#', 'F#', 'C', 'F', 'G', 'E', 'D#', 'G#', 'D', 'A#', 'A', 'B'],
      dtype=object)

In [10]:
# dummies = pd.get_dummies(df['key'])
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [11]:
dfle = df
dfle.key = le.fit_transform(dfle.key)
df

,acousticness,duration_ms,energy,instrumentalness,key,mode,speechiness,tempo,valence
0,0.61100,99373,0.910,0.000000,4,1,0.0525,166.969,0.814
1,0.24600,137373,0.737,0.000000,9,0,0.0868,174.003,0.816
2,0.95200,170267,0.131,0.000000,3,0,0.0362,99.488,0.368
3,0.70300,152427,0.326,0.000000,4,1,0.0395,171.758,0.227
4,0.95000,82625,0.225,0.123000,8,1,0.0456,140.576,0.390
...,...,...,...,...,...,...,...,...,...
232720,0.00384,326240,0.714,0.544000,5,1,0.0316,115.542,0.962
232721,0.03290,282447,0.683,0.000880,7,0,0.0337,113.830,0.969
232722,0.90100,166960,0.419,0.000000,5,1,0.1480,84.135,0.813
232723,0.26200,222442,0.704,0.000000,0,1,0.1460,100.031,0.489


In [12]:
x = df

In [13]:
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
minmax_df = scaler.fit_transform(x)
x = pd.DataFrame(minmax_df, columns = x.columns)

In [14]:
x

,acousticness,duration_ms,energy,instrumentalness,key,mode,speechiness,tempo,valence
0,0.613454,0.015167,0.910909,0.000000,0.363636,1.0,0.032070,0.642704,0.814
1,0.246988,0.022029,0.737732,0.000000,0.818182,0.0,0.068374,0.675801,0.816
2,0.955823,0.027969,0.131113,0.000000,0.272727,0.0,0.014818,0.325182,0.368
3,0.705823,0.024747,0.326313,0.000000,0.363636,1.0,0.018311,0.665238,0.227
4,0.953815,0.012142,0.225209,0.123123,0.727273,1.0,0.024767,0.518516,0.390
...,...,...,...,...,...,...,...,...,...
232720,0.003855,0.056136,0.714709,0.544545,0.454545,1.0,0.009949,0.400722,0.962
232721,0.033032,0.048227,0.683677,0.000881,0.636364,0.0,0.012172,0.392666,0.969
232722,0.904618,0.027372,0.419408,0.000000,0.454545,1.0,0.133150,0.252941,0.813
232723,0.263052,0.037391,0.704699,0.000000,0.000000,1.0,0.131033,0.327737,0.489


In [15]:
y = pd.DataFrame(y)

In [16]:
scaler = preprocessing.MinMaxScaler()
y = scaler.fit_transform(y)
y = pd.DataFrame(y,columns = ['popularity'])

In [17]:
y

,popularity
0,0.00
1,0.01
2,0.03
3,0.00
4,0.04
...,...
232720,0.39
232721,0.38
232722,0.47
232723,0.44


**Training of our model**

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 420)

In [48]:
X_train.shape

(186180, 9)

In [20]:
y_test

,popularity
182914,0.26
110891,0.70
55126,0.56
170980,0.20
183122,0.21
...,...
75338,0.01
99259,0.54
226188,0.60
189845,0.19


In [21]:
X_test

,acousticness,duration_ms,energy,instrumentalness,key,mode,speechiness,tempo,valence
182914,0.452811,0.035590,0.414403,0.000000,0.454545,1.0,0.020428,0.355983,0.4580
110891,0.042871,0.040157,0.866864,0.000140,0.909091,0.0,0.020322,0.360628,0.3510
55126,0.101406,0.032941,0.449438,0.000000,0.454545,1.0,0.404107,0.639231,0.0396
170980,0.878514,0.051823,0.902901,0.000001,0.636364,1.0,0.816892,0.480383,0.0328
183122,0.488956,0.029804,0.817814,0.000036,0.272727,1.0,0.015982,0.445192,0.9700
...,...,...,...,...,...,...,...,...,...
75338,0.663655,0.019253,0.467457,0.000045,0.909091,1.0,0.032388,0.242815,0.9640
99259,0.000021,0.051855,0.933933,0.000009,0.818182,0.0,0.013548,0.379096,0.4350
226188,0.289157,0.046931,0.510501,0.000053,0.727273,0.0,0.002646,0.468192,0.3270
189845,0.077912,0.040291,0.808805,0.000035,0.909091,1.0,0.055356,0.445098,0.8280


**To check the best ML model for our prediction making we use to check MAE and MSE for some different regression models. And  based upon that we are going to finalize which model to use for the prediction.**

# Logistic Regression 

In [38]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(X_train, y_train)

LinearRegression()

In [40]:
y_pred = reg.predict(X_test)

In [41]:
from sklearn import metrics

print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error (MSE):', metrics.mean_squared_error(y_test, y_pred))

Mean Absolute Error (MAE): 0.13236232156242714
Mean Squared Error (MSE): 0.027278436592226663


# RandomForestRegressor

In [25]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor()
regr.fit(X_train, y_train)

RandomForestRegressor()

In [26]:
import joblib
joblib.dump(regr,"./model.joblib")

['./model.joblib']

In [36]:
y_pred1=regr.predict(X_test)

In [37]:
from sklearn import metrics

print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error (MSE):', metrics.mean_squared_error(y_test, y_pred))


Mean Absolute Error (MAE): 0.08556432591772922
Mean Squared Error (MSE): 0.014667317995020104


# KNeighbors Regressor

In [42]:
from sklearn.neighbors import KNeighborsRegressor
neigh = KNeighborsRegressor(n_neighbors=2)
neigh.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=2)

In [43]:
y_pred = neigh.predict(X_test)

In [44]:
from sklearn import metrics

print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error (MSE):', metrics.mean_squared_error(y_test, y_pred))

Mean Absolute Error (MAE): 0.11343355892147385
Mean Squared Error (MSE): 0.025592983671715547


**From the above three models ,it can be concluded that the RANDOMFOREST REGRESSOR has been the best model for the prediction.It has the lowest Mean Absolute Error(MAE) and lowest Mean Squared Error(MSE).**